## Koopman Model Predictive Control for flow control

This tutorial repeats a similar setting of the paper (mainly Section 5.1) and attached respositpory

* Paper: A data-driven Koopman model predictive control framework for nonlinear flows https://arxiv.org/pdf/1804.05291.pdf
* Supplementary code: https://github.com/arbabiha/KoopmanMPC_for_flowcontrol

Note that there are discrepancies between code and desciption in paper. While these are not critical, we highlight discrepancies to the paper (which are to some extent also in supplementary code).  

The plant model is a one-dimensional Burger equation with periodic boundaries.

$$
\frac{\partial v}{\partial t} + v \frac{\partial v}{\partial x} = \nu \frac{\partial v^2}{\partial^2 x} + f(x,t)
$$

with 

$$
x \in [0, 2\pi] \\
v(0, t) = v(2\pi, t) \\ 
t \in [0, \infty]
$$

*In the paper the domain is $x \in [0, 1]$, while the our choice matches the setting in the supplementary code.*

There are two control parameters $u=(u_1, u_2) \in \mathbb{R}^2$ with which we can control the system state in order to move the system to a reference solution. The control parameters affect the entire state:

$$
\begin{align}
f(x,t) &= u_1(t) f_1(x) + u_2(t)f_2(x)\\
&= u_1(t) \exp(-((15/(2\pi) (x - \pi/2))^2)) + u_2(t) \exp(-((15/(2\pi) (x - 3/2\pi))^2)))
\end{align}
$$

with constrains

$$
-0.1 \leq u_{1,2} \leq 0.1
$$

*(The function parameters are adapted to the larger domain $x \in [0, 2\pi]$ in contrast to the paper.)*


An initial condition of the system is specified with

$$
v(x, 0) = a \exp(-(((x - \pi) * 5/(2\pi))^2)) + (1 - a) \sin(2x)^2
$$

*(The function parameters are adapted to the larger domain $x \in [0, 2\pi]$ in contrast to the paper.)*


The control objective is to follow the reference state

$$
\begin{align}
v_{ref} (x, 0 \leq t \leq 20) &= 0.5\\
v_{ref} (x, 20 \leq t \leq 40) &= 1\\
v_{ref} (x, 40 \leq t \leq 60) &= 0.5
\end{align}
$$

*Note that in the paper the time sampling is 0.01 to solve the system, but for the data collection only every 19th state is taken, which leads to an effective time interval of $\Delta t = 0.19$. Therefore, the time intervals in the reference are adapted such that they roughly match the setting in the paper.*


To obtain a suitable control sequence, we identify the system dynamics with the Extended Dynamic Mode Decomposition (EDMD) and generate a control sequence with linear Model Predictive Control (MPC). 

In [1]:
import matplotlib.pyplot as plt

from IPython.display import HTML
import numpy as np
import pandas as pd
from matplotlib.animation import FuncAnimation
from scipy.interpolate import interp1d
from scipy.io import loadmat
from sklearn.base import BaseEstimator
from tqdm import tqdm
from datafold import (
    EDMD,
    DMDControl,
    TSCColumnTransformer,
    TSCDataFrame,
    TSCIdentity,
    TSCTakensEmbedding,
    TSCTransformerMixin,
)
from datafold.appfold.kmpc import LinearKMPC
from datafold.utils._systems import Burger1DPeriodicBoundary

In [2]:
rng = np.random.default_rng(2)

### Data collection

#### Original system and 

In [3]:
sys = Burger1DPeriodicBoundary(nu=0.01)

# control functions
f1 = np.atleast_2d(np.exp(-((15/(2*np.pi) * (sys.x_nodes - 0.5*np.pi)) ** 2)))
f2 = np.atleast_2d(np.exp(-((15/(2*np.pi) * (sys.x_nodes - 1.5*np.pi)) ** 2)))

# initial condition function
ic1 = np.exp(-(((sys.x_nodes - 2*np.pi*0.5) * 5/(2*np.pi)) ** 2))
ic2 = np.sin(2 * sys.x_nodes) ** 2
icfunc = lambda a: a * ic1 + (1 - a) * ic2

#### Sampling parameters

In EDMD we do not take the full state but only every 10th node on the domain.

In [4]:
# time series sampling options
dt = 0.19
sim_length = 200
training_size = 100

# function to subselect state measurements to every 10th node
def subselect_measurements(tscdf):
    return tscdf.iloc[:, 9::10]

# control options
umin, umax = (-0.1, 0.1)

In [5]:
time_values = np.arange(0, dt * sim_length + 1e-12, dt)

### Perform sampling

Fill lists with time series, which are then described with *datafold*'s data structure `TSCDataFrame`.


In [6]:
X_tsc = []
U_tsc = []

for i in tqdm(range(training_size)):
    # sample a new initial condition
    ic = icfunc(rng.uniform(0, 1))  
    
    # sample a random control sequence
    rand_vals = rng.uniform(umin, umax, size=(len(time_values), 2))
    
    # set up control function from the two control inputs
    U1rand = lambda t: np.atleast_2d(interp1d(time_values, rand_vals[:, 0], kind="previous")(t)).T
    U2rand = lambda t: np.atleast_2d(interp1d(time_values, rand_vals[:, 1], kind="previous")(t)).T
        
    def U(t, x):
        # while we have 2 effective control inputs they affect each state coordinate according to this function 
        return U1rand(t) * f1 + U2rand(t) * f2
    
    # perform system prediction
    X_predict, Ufull = sys.predict(
        ic, U=U, time_values=time_values, require_last_control_state=False
    )

    # drop last control input, because for the last state no prediction is performed
    U = TSCDataFrame.from_array(
        rand_vals[:-1, :],
        time_values=Ufull.time_values(),
        feature_names=["u1", "u2"],
    )

    X_tsc.append(X_predict)
    U_tsc.append(U)

# finalize the time series collection data
X_tsc = TSCDataFrame.from_frame_list(X_tsc)
U_tsc = TSCDataFrame.from_frame_list(U_tsc)

100%|█████████████████████████████████████████| 100/100 [00:26<00:00,  3.82it/s]


In [7]:
X_tsc.head(5)

feature        x0        x1        x2        x3        x4        x5        x6  \
ID time                                                                         
0  0.00  0.000505  0.012482  0.047402  0.103049  0.175877  0.261248  0.353723   
   0.19  0.006719  0.020279  0.049590  0.093109  0.147763  0.210579  0.278910   
   0.38  0.013534  0.025553  0.050389  0.085817  0.129411  0.179012  0.232879   
   0.57  0.021277  0.029966  0.050732  0.080297  0.116383  0.157253  0.201584   
   0.76  0.032504  0.034800  0.051237  0.076090  0.106645  0.141263  0.178820   

feature        x7        x8        x9  ...       x90       x91       x92  \
ID time                                ...                                 
0  0.00  0.447418  0.536375  0.614951  ...  0.614951  0.536375  0.447418   
   0.19  0.350355  0.422632  0.493428  ...  0.714647  0.666874  0.588267   
   0.38  0.289600  0.347978  0.406918  ...  0.720190  0.725388  0.705878   
   0.57  0.248367  0.296807  0.346237  ...  0.662216  0.692435  0.712929   
   0.76  0.218506  0.259708  0.301943  ...  0.592642  0.628560  0.661008   

feature       x93       x94       x95       x96       x97       x98       x99  
ID time                                                                        
0  0.00  0.353723  0.261248  0.175877  0.103049  0.047402  0.012482  0.000505  
   0.19  0.481991  0.360134  0.240733  0.140171  0.067735  0.025047  0.007445  
   0.38  0.646697  0.530618  0.369256  0.213341  0.103245  0.042452  0.016947  
   0.57  0.719328  0.696787  0.590587  0.380482  0.183554  0.074708  0.031057  
   0.76  0.687498  0.708114  0.726475  0.652629  0.392981  0.160064  0.059943  

[5 rows x 100 columns]

In [8]:
U_tsc.head(5)

feature        u1        u2
ID time                    
0  0.00 -0.040302  0.062845
   0.19 -0.081617  0.020020
   0.38  0.045712 -0.062420
   0.57 -0.088971 -0.045006
   0.76  0.031487  0.012453

In [24]:
tsid = 0  # can select time series to plot

f, ax = plt.subplots(figsize=(8, 7), nrows=2)
plt.close()  # close for animation, the figure still exists

(ref_line,) = ax[0].plot(sys.x_nodes, X_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[0].to_numpy(), label="model")
ax[0].legend(loc="upper left")

def Ufunc(u):
    return u[0] * f1 + u[1] * f2

(control_line,) = ax[1].plot(sys.x_nodes,
                       Ufunc(U_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[0].to_numpy()).ravel(),
                       label="random control input")
ax[1].set_ylim(-0.1, 0.1)
ax[1].legend(loc="upper left")

def func(i):
    ref_line.set_ydata(X_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[i, :].to_numpy())
    vals = U_tsc.loc[pd.IndexSlice[tsid, :], :].iloc[i, :].to_numpy()
    control_line.set_ydata(Ufunc(vals))
    return (ref_line, control_line, )

anim = FuncAnimation(f, func=func, frames=U_tsc.n_timesteps);
HTML(anim.to_html5_video())

## Set up system identification with EDMD and control input

For the data to train the EDMD, we use a reduced number of nodes and also attach the control input to the actual system states. For this, however, we shift the time index by one in the control input to match the control input (in the past) and resulted system state. 

Because of the time shift the initial state has not matching ctontrol input. We fill this with zeros. These values are ignored later anyway when we set up the EDMD dictionary.

In [10]:
def shift_time_index_U(_X, _U):
    new_index = _X.groupby("ID").tail(_X.n_timesteps - 1).index
    return _U.set_index(new_index)

In [11]:
X_tsc_reduced = subselect_measurements(X_tsc)

X_tsc_reduced = pd.concat(
    [X_tsc_reduced, shift_time_index_U(X_tsc_reduced, U_tsc)], axis=1
).fillna(0)

In [12]:
X_tsc_reduced.head(5)

feature        x9       x19       x29       x39       x49       x59       x69  \
ID time                                                                         
0  0.00  0.614951  0.352659  0.284394  0.894827  0.264416  0.852158  0.381333   
   0.19  0.493428  0.471498  0.224810  0.888657  0.293167  0.706928  0.551812   
   0.38  0.406918  0.629393  0.183127  0.784118  0.357912  0.590388  0.805753   
   0.57  0.346237  0.717336  0.166435  0.673454  0.530439  0.507621  0.885717   
   0.76  0.301943  0.684019  0.141133  0.583136  0.838769  0.448508  0.832479   

feature       x79       x89       x99        u1        u2  
ID time                                                    
0  0.00  0.265853  0.678180  0.000505  0.000000  0.000000  
   0.19  0.225140  0.734218  0.007445 -0.040302  0.062845  
   0.38  0.197675  0.698197  0.016947 -0.081617  0.020020  
   0.57  0.170645  0.625450  0.031057  0.045712 -0.062420  
   0.76  0.151501  0.554260  0.059943 -0.088971 -0.045006

We set up a custom data transformation, which can be used in the data pipeline.

In [13]:
class L2Norm(BaseEstimator, TSCTransformerMixin):
    def fit(self, X):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["l2norm"]

    def transform(self, X: TSCDataFrame, y=None):
        return TSCDataFrame.from_same_indices_as(
            X,
            np.sum(np.square(np.abs(X.to_numpy())), axis=1) / X.shape[1],
            except_columns=self.get_feature_names_out(),
        )

### Set up controlled EDMD by specifying dictionary underlying DMD model

In [14]:
l2norm = ("l2_x", L2Norm(), lambda df: df.columns.str.startswith("x"))

delay1 = (
    "delay_x",
    TSCTakensEmbedding(delays=4),
    lambda df: df.columns.str.startswith("x"),
)
delay2 = (
    "delay_u",
    TSCTakensEmbedding(delays=3),
    lambda df: df.columns.str.startswith("u"),
)

_dict = (
    "tde",
    TSCColumnTransformer([l2norm, delay1, delay2], verbose_feature_names_out=False),
)

_id = ("_id", TSCIdentity(include_const=True))

dict_steps = [_dict, _id]


# It is essential to use DMD with control
edmd = EDMD(dict_steps, dmd_model=DMDControl(), include_id_state=False)
edmd

EDMD(dict_steps=[('tde',
                  TSCColumnTransformer(transformers=[('l2_x', L2Norm(),
                                                      <function <lambda> at 0x7fa5da3508b0>),
                                                     ('delay_x',
                                                      TSCTakensEmbedding(delays=4),
                                                      <function <lambda> at 0x7fa5da350b80>),
                                                     ('delay_u',
                                                      TSCTakensEmbedding(delays=3),
                                                      <function <lambda> at 0x7fa5da350c10>)],
                                       verbose_feature_names_out=False)),
                 ('_id', TSCIdentity(include_const=True))],
     dmd_model=DMDControl(), include_id_state=False)

Train the model with sampled data. The flag `dict_preserves_id_states=True` indicates that the original states are also contained in the dictionary, which makes the inverse mapping from dictionary states to full states easier. 

In [15]:
edmd.fit(X_tsc_reduced, U=U_tsc, dict_preserves_id_states=True);

Sample of the dictionary states in EDMD

In [16]:
edmd.transform(X_tsc_reduced.head(9))

feature    l2norm        x9       x19       x29       x39       x49       x59  \
ID time                                                                         
0  0.76  0.285048  0.301943  0.684019  0.141133  0.583136  0.838769  0.448508   
   0.95  0.254096  0.268404  0.632522  0.133444  0.512709  0.873360  0.405392   
   1.14  0.224823  0.242066  0.571858  0.119214  0.457050  0.794670  0.374325   
   1.33  0.231303  0.220925  0.525406  0.124385  0.412456  0.731506  0.357196   
   1.52  0.226133  0.203548  0.484013  0.156189  0.376019  0.669866  0.381356   

feature       x69       x79       x89  ...    x99:d4        u1        u2  \
ID time                                ...                                 
0  0.76  0.832479  0.151501  0.554260  ...  0.000505 -0.088971 -0.045006   
   0.95  0.757625  0.143187  0.494160  ...  0.007445  0.031487  0.012453   
   1.14  0.684959  0.141168  0.444787  ...  0.016947 -0.069988 -0.013474   
   1.33  0.624040  0.190619  0.404132  ...  0.031057  0.033859 -0.015443   
   1.52  0.581099  0.541359  0.370507  ...  0.059943  0.026637  0.093487   

feature     u1:d1     u2:d1     u1:d2     u2:d2     u1:d3     u2:d3  const  
ID time                                                                     
0  0.76  0.045712 -0.062420 -0.081617  0.020020 -0.040302  0.062845      1  
   0.95 -0.088971 -0.045006  0.045712 -0.062420 -0.081617  0.020020      1  
   1.14  0.031487  0.012453 -0.088971 -0.045006  0.045712 -0.062420      1  
   1.33 -0.069988 -0.013474  0.031487  0.012453 -0.088971 -0.045006      1  
   1.52  0.033859 -0.015443 -0.069988 -0.013474  0.031487  0.012453      1  

[5 rows x 60 columns]

## Set up Model Predictive Control 

Prediction parameters:

In [17]:
Tpred = dt * 20  # prediction horizon
horizon = int(np.round(Tpred // dt))
Tend = 60
Nsim = int(Tend // dt) + 1

Perform initial prediction with no control applied for as many time steps are needed for to evaluate the EDMD dictionary (attribute `edmd.n_samples_ic_`. 

In [18]:
ic = icfunc(0.2)  # can also be adapted to control another time series

X_init, _ = sys.predict(
    ic,  
    U=np.zeros((4, sys.n_control_in_)),
    time_values=np.arange(0, edmd.n_samples_ic_ * dt, dt),
    require_last_control_state=True,
)

In [19]:
kmpc = LinearKMPC(
    edmd=edmd,
    horizon=horizon,
    state_bounds=None,
    input_bounds=np.array([[-0.1, 0.1], [-0.1, 0.1]]),
    qois=X_tsc_reduced.columns[X_tsc_reduced.columns.str.startswith("x")],
    cost_running=1,
    cost_terminal=1,
    cost_input=1,
)

Reference time series (full and reduced).

In [20]:
start_time = X_init.time_values()[-1]
time_values_ref = np.arange(0, start_time + Tend, dt)
X_ref = np.zeros(len(time_values_ref))
X_ref[time_values_ref <= 20] = 0.5
X_ref[np.logical_and(time_values_ref > 20, time_values_ref < 40)] = 1
X_ref[time_values_ref > 40] = 0.5
X_ref = np.outer(X_ref, np.ones(X_tsc.shape[1]))
X_ref = TSCDataFrame.from_array(
    X_ref, time_values=time_values_ref, feature_names=X_tsc.columns
)

X_ref_reduced = subselect_measurements(X_ref)

U_ic = TSCDataFrame.from_array(
    np.zeros((5, 2)), time_values=X_init.time_values(), feature_names=["u1", "u2"]
)

### Perform simulation for the initial time embedding with no control

In [21]:
# keep track of current EDMD and model state
edmd_state = pd.concat([subselect_measurements(X_init), U_ic], axis=1)
model_state = X_init.iloc[[-1], :]

X_model_evolution = X_init
U_evolution = U_ic

# record the uncontrolled time series to compare
X_model_unctr_evolution = X_init.copy()

# system error between state and reference time series
X_error_evolution = X_init - X_ref.iloc[[0], :].to_numpy()

for i in tqdm(range(X_init.shape[0], Nsim)):

    # start horizon from next state (we can't change the current state)
    reference = X_ref_reduced.iloc[i: i + horizon, :]

    t = X_model_evolution.time_values()[-1]
    t_new = X_model_evolution.time_values()[-1] + dt

    if reference.shape[0] != kmpc.horizon:
        # stop if the rest of reference signal is smaller than horizon
        break

    U = kmpc.control_sequence(edmd_state, reference=reference)

    # use only the first control input and optimize again for the next window
    Ufull = U.iloc[0, 0] * f1 + U.iloc[0, 1] * f2

    X_model, _ = sys.predict(X_model_evolution.iloc[[-1], :], U=Ufull, time_values=np.array([t, t_new]))
    X_model_evolution = pd.concat([X_model_evolution, X_model.iloc[[1], :]], axis=0)

    diff = X_model.iloc[[1], :] - X_ref.iloc[[i], :].to_numpy()
    X_error_evolution = pd.concat([X_error_evolution, diff])

    X_model_unctr, _ = sys.predict(X_model_unctr_evolution.iloc[[-1], :], U=np.zeros_like(sys.x_nodes)[np.newaxis, :], time_values=np.array([t, t_new]))
    X_model_unctr_evolution = pd.concat([X_model_unctr_evolution, X_model_unctr.iloc[[1], :]], axis=0)

    U_evolution = pd.concat([U_evolution, U.iloc[[0], :]], axis=0)

    # prepare new edmd state
    X_model_last = subselect_measurements(X_model_evolution.iloc[-edmd.n_samples_ic_ :, :])
    U_last = U_evolution.iloc[-edmd.n_samples_ic_ :-1, :]
    U_last_shifted = shift_time_index_U(X_model_last, shift_time_index_U(X_model_last, U_last))
    edmd_state = pd.concat([X_model_last, U_last_shifted], axis=1).fillna(0)

 95%|███████████████████████████████████████▏ | 297/311 [00:38<00:01,  7.72it/s]


In [23]:
f, ax = plt.subplots(figsize=(8, 7), nrows=3, sharex=True)
plt.close() # see https://stackoverflow.com/a/47138474 (first comment)
(model_line,) = ax[0].plot(sys.x_nodes, X_model_evolution.iloc[0], label="model controlled")
(model_uctr_line,) = ax[0].plot(sys.x_nodes, X_model_unctr_evolution.iloc[0], label="model uncontrolled")
(ref_line,) = ax[0].plot(sys.x_nodes, X_ref.iloc[0], label="reference")
ax[0].legend(loc="upper left")
ax[0].set_ylim(0, 1.3)

Ufunc = lambda u, x: (u[0] * f1 + u[1] * f2).ravel()
(control_line,) = ax[1].plot(
    sys.x_nodes,
    Ufunc(U_evolution.iloc[0, :].to_numpy(), None),
    label="control",
)
ax[1].set_ylim(umin, umax)
ax[1].legend(loc="upper left")

(error_line,) = ax[2].plot(sys.x_nodes, X_error_evolution.iloc[0, :].to_numpy(), c="red", label="difference")
ax[2].legend(loc="upper left")

def func(i):
    model_line.set_ydata(X_model_evolution.iloc[i, :].to_numpy())
    model_uctr_line.set_ydata(X_model_unctr_evolution.iloc[i, :].to_numpy())
    ref_line.set_ydata(X_ref.iloc[i, :].to_numpy())
    control_line.set_ydata(Ufunc(U_evolution.iloc[i].to_numpy(), None))
    error_line.set_ydata(X_error_evolution.iloc[i, :].to_numpy())

    return (
        model_line,
        model_uctr_line,
        ref_line,
        error_line
    )

anim = FuncAnimation(f, func=func, frames=U_evolution.shape[0]);
HTML(anim.to_html5_video())